### 1. 데이터 생성하기
(1) ./datasets 아래에 .hwp 파일을 모두 load하기

(2) text_splitter로 분할하기

(3) all_docs에 모든 chunk들 append하기

(4) Pineone Vector database에 업로드하기

In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_teddynote.document_loaders.hwp import HWPLoader
import os

document_list = []

# text splitter 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=650,
    chunk_overlap=100,
)

# datasets 폴더를 시작점으로 하위 폴더까지 재귀적으로 순회
for root, dirs, files in os.walk("./datasets"):
    for file in files:
        if file.endswith(".hwp"):
            file_path = os.path.join(root, file)
            loader = HWPLoader(file_path)
            # 1) 문서 로드
            docs = loader.load()
            # 2) 텍스트 스플리터로 분할
            split_docs = text_splitter.split_documents(docs)
            document_list.extend(split_docs)

print(f"The number of Splitted Chunks: {len(document_list)}")

The number of Splitted Chunks: 2332


In [21]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [22]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'ds-chatbot'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

### 2. 답변 생성을 위한 Retrieval 및 Prompt 활용하기
(1) RetrievalQA에 전달하기 위해 retriever 생성

(2) LangChain에서 제공하는 rlm/rag-prompt 사용

In [25]:
query = '장학금은 무엇이 있나요?'
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(id='4b0733f7-c6d3-4e57-8468-d901645d682a', metadata={'source': './datasets/3편 학사행정/4장 교학행정/4. 대학원(특수)장학금지급에관한내규(20230710)_일부개정.hwp'}, page_content='[별표1] 장학금 지급기준 및 금액 <신설 2017.6.14., 개정 2019.10.2., 2020.7.1., 2021.4.14., 2022.1.12., 2022.9.27., 2023.7.10.>구분장학금 명칭지급기준장학금액비고교육대학원 및상담‧산업대학원현직교원우대장학금- 교육대학원(영양교육전공제외), 상담․산업대학원 · 유치원, 초․중․고등학교에 재직 중이거나 어린이집에 재직 중인 교사- 영양교육전공· 유치원, 초․중․고등학교에 재직 중이거나 어린이집에 재직 중인 교사 및 영양사수업료의\xa015%교육대학원 및 상담‧산업대학원성적우수장학금- 재학생 5명 이상   전공직전학기 성적 최우수인 자 수업료의 20%직전학기 성적 상위 10% 이내로서 최우수자를 제외한 자수업료의 15%- 재학생 5명 미만   전공직전학기 성적 최우수인 자수업료의 15%교육대학원 및상담‧산업대학원덕성동문장학금- 학부 졸업 후 1년 이내 진학자수업료의 50%·2학기 지급·입학금 제외교육대학원및상담‧산업대학원원우회장학금- 각 전공별 원우회장300,000원교육대학원및상담‧산업대학원교외장학금- 정부 또는 지방자치단체에서 지급하는 장학금- 사회단체, 기업체, 개인이 지급하는 장학금지정금액상담‧산업대학원교류협력기관장학금- 교외기관과의 교류협력체결에 따라 협력기관에서 추천한 자수업료의'),
 Document(id='88eac6a1-29d4-4a57-a03f-71eecab8b7c8', metadata={'source': './datasets/3편 학사행정/4장 교학행정/장학금규정시행세

In [26]:
from langchain import hub
from langchain_openai import ChatOpenAI

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model='gpt-4o')

### 3. 답변 생성하기
(1) RetrievalQA를 통해 LLM에 전달

In [27]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

ai_message = qa_chain.invoke({"query": query})

ai_message

{'query': '\x08장학금은 무엇이 있나요?',
 'result': '장학금은 다양한 기준에 따라 여러 종류가 있습니다. 예를 들어, 성적 우수 장학금, 가정형편이 곤란한 학생을 위한 가계곤란 장학금, 그리고 외국인 학생을 위한 장학금 등이 있습니다. 각각의 장학금은 지급 기준과 금액이 다르며, 성적, 재직 여부, 한국어 능력 등을 기준으로 선정됩니다.'}